# Twitter scraping goes BOUM

Problème de RAM, il fait enregistrer en csv pour chaque compte et concat tout ça. 

## Import 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import datetime

import seaborn as sns

from os import listdir
from os.path import isfile, join

#Parsing:
import dateparser # $ pip install dateparser

## List candidates

In [2]:
twitter_candidates={#'zemmour': ['@ZemmourEric', '@Reconquete2022'], 
                    #'le_pen':['@MLP_officiel', '@RNational_off'], 
                   #'roussel': ['@Fabien_Roussel', '@PCF'], 
                   #'melenchon' : ['@JLMelenchon', '@FranceInsoumise'], 
                   #'jadot':['@yjadot', '@EELV'], 
                   ##'hidalgo':['@Anne_Hidalgo', '@partisocialiste'], 
                   'pecresse':['@vpecresse', '@lesRepublicains'], 
                   'poutou':['@PhilippePoutou', '@NPA_officiel'], 
                   'arthaud':['@n_arthaud', '@LutteOuvriere'],
                   'lassale':['@jeanlassalle'], 
                   'dupont_aignan':['@dupontaignan', '@DLF_Officiel']}

## Retrieving tweets using Minet

In [4]:
import minet
from minet.twitter import TwitterAPIScraper
scraper = TwitterAPIScraper()

**Changement:**


On avait `min_retweets` à 100, on va essayer de le mettre à 10. 

In [5]:
def retrieve_tweets_cont_username(username):
    """
    @Inputs: username, str
    
    @Outputs: list of tweets
    """

    #Initialising a temp list:
    l_temp = []
    #Retriving the tweets containing the username:
    
    query = str('('+username+') min_retweets:10 until:2022-03-14 since:2022-01-01')
    #(#zemmour2022) min_retweets:100 until:2022-03-15 since:2022-01-01'):
    
    for tweet in scraper.search_tweets(query):
        l_temp.append(tweet)
    return l_temp


def create_df_from_tweets(l_temp, candidate):
    """
    @Inputs: 
        - l_temp, list of tweets (raw)
    
    @Outputs: 
        - df, clean, with tweets and candidate
    """
    
    
    #We are using the keys not containing any list:
    list_keys=['id', 'local_time', 'timestamp_utc', 'text', 'url', 'quoted_id', 'quoted_user', 'quoted_user_id', 
               'quoted_timestamp_utc', 'retweeted_id', 'retweeted_user', 'retweeted_user_id', 'retweeted_timestamp_utc', 
               'links_to_resolve', 'collection_time', 'match_query', 'coordinates', 
               'to_tweetid', 'to_username', 'to_userid', 'lang', 'possibly_sensitive', 'retweet_count', 'like_count', 
               'reply_count', 'user_id', 'user_screen_name', 'user_name', 'user_friends', 'user_followers', 
               'user_location', 'user_verified', 'user_description', 'user_created_at', 'user_tweets', 'user_likes', 
               'user_lists', 'user_image', 'user_url', 'user_timestamp_utc', 'source_url', 'source_name']
    
    #Init an empty dataframe:
    df_ = pd.DataFrame()
    #Loop over the raw tweets in l_temp
    for el in l_temp:
        #Creating an empty dict
        dict_temp={}
        #Looping over the selected keys
        for k in list_keys:
            #If the key exist, we retrieve the element 
            try:
                dict_temp[k]=el[k]
            #Else
            except:
                #do nothing
                pass
        #Creating an empty dataframe of only one row (the tweet we are looping over from el in df_temp)
        df_temp=pd.DataFrame(data=[dict_temp])
        #Concat the previously treated tweets and the new one
        df_=pd.concat([df_, df_temp])
        
    #Indicating the candidate's name:
    df_['candidate']=candidate
    
    #We can return the DataFrame:
    return df_

In [6]:
def main_tweets():
    for candidate in twitter_candidates.keys():
        df_tweets = pd.DataFrame()
        print('-----------------------')
        print(candidate)
        for username in twitter_candidates[candidate]:
            print(username)
            l_temp = retrieve_tweets_cont_username(username)
            print(len(l_temp))
            df_ = create_df_from_tweets(l_temp, candidate)

            
            
            df_tweets = pd.concat([df_tweets, df_])
            
            
            # !!!!! On ne concat plus sur un gros df, je crois que ça créé un problème de RAM.
            #On save directement en CSV
            
        df_tweets.to_csv('data/tweets/10/df_'+candidate+'.csv')
    #return df_tweets
    return True

Ca a pris 8min pour 100 rt min 

combien pour 10 ? 

On commence à 
19:30

In [7]:
%time df_tweets = main_tweets()

-----------------------
pecresse
@vpecresse
11972
@lesRepublicains
2836
-----------------------
poutou
@PhilippePoutou
462
@NPA_officiel
71
-----------------------
arthaud
@n_arthaud
237
@LutteOuvriere
46
-----------------------
lassale
@jeanlassalle
172
-----------------------
dupont_aignan
@dupontaignan
1160
@DLF_Officiel
43
Wall time: 20min 15s
